#  데이터 다듬기와 가공 (Pandas 심화)

## 학습 목표
이 토픽을 수강한 뒤, 수강생은 다음을 할 수 있어야 합니다:

- 데이터 프레임과 시리즈를 탐색하고 설정할 수 있다.  
- 결측값, 중복값, 이상점을 처리할 수 있다.  
- 문자 및 숫자 데이터를 가공할 수 있다.  
- 날짜와 시간 데이터를 다룰 수 있다.  
- 데이터를 합치고 그룹화하여 분석할 수 있다.


# 0. 들어가기

데이터 분석에서 가장 중요한 첫 단계는 **데이터 클리닝(data cleaning)** 입니다.  
- 결측값, 중복값, 이상점을 처리하지 않으면 잘못된 결론에 도달할 수 있습니다.  
- 깨끗하고 구조화된 데이터만이 **믿을 수 있는 통계와 모델**을 만들어 줍니다.  

👉 이번 파트에서는 Pandas를 활용해 DataFrame을 다루는 기초부터 시작합니다.  

# 1. DataFrame 기본기

## 1.1 DataFrame과 Series
- **Series**: 1차원 데이터 (열 하나)
- **DataFrame**: 2차원 데이터 (행 + 열)

In [ ]:
import pandas as pd

# Series 예시
ages = pd.Series([22, 35, 58, 42])
print(ages)

# DataFrame 예시
data = {
    "Name": ["Alice", "Bob", "Charlie", "Diana"],
    "Age": [22, 35, 58, 42],
    "Gender": ["female", "male", "male", "female"]
}
df = pd.DataFrame(data)
df

## 1.2 데이터 탐색 함수
- `head()` : 상위 5행 미리보기
- `tail()` : 하위 5행 미리보기
- `shape` : (행 개수, 열 개수)
- `columns` : 열 이름 리스트
- `info()` : 데이터 타입과 결측치 확인
- `describe()` : 기초 통계량 요약

In [ ]:
import pandas as pd

# Titanic 데이터 불러오기
titanic = pd.read_csv("data/titanic.csv")

# Bike Sharing 데이터 불러오기
bike = pd.read_csv("data/bike_sharing.csv")

print(titanic.head())
print(titanic.tail())
print(titanic.shape)
print(titanic.columns)
print(titanic.info())
print(titanic.describe())

## 1.3 인덱스와 컬럼 이름 설정
- `set_index("컬럼명")`: 특정 열을 인덱스로 지정
- `reset_index()`: 기존 인덱스를 다시 숫자형으로 리셋
- `rename(columns={...})`: 열 이름 변경

In [ ]:
titanic2 = titanic.set_index("Name")
print(titanic2.head())

titanic2 = titanic2.reset_index()
print(titanic2.head())

titanic3 = titanic.rename(columns={"Sex": "Gender"})
print(titanic3.head())

## 1.4 데이터 타입 변환
- `astype()` : 데이터 타입 변경

In [ ]:
# Age를 float형으로 변환
titanic["Age"] = titanic["Age"].astype(float)
print(titanic.info())

## 1.5 불린 인덱싱 (조건으로 행 선택)
- 특정 조건을 만족하는 행만 선택할 수 있음
- 여러 조건은 **논리 연산자**(`&`, `|`, `~`)로 조합

In [ ]:
# 나이가 30 이상인 행 선택
print(titanic[titanic["Age"] >= 30])

# 성별이 female이고 나이가 30 이상인 경우
print(titanic[(titanic["Sex"] == "female") & (titanic["Age"] >= 30)])

## 1.6 행과 열 삭제
- `drop("열이름", axis=1)`: 열 삭제
- `drop(행인덱스, axis=0)`: 행 삭제

In [ ]:
# 열 삭제
titanic_drop_col = titanic.drop("Sex", axis=1)
print(titanic_drop_col.head())

# 행 삭제 (0번째 행)
titanic_drop_row = titanic.drop(0, axis=0)
print(titanic_drop_row.head())

### ✅ 체크포인트
- Pandas의 기본 단위는 **Series(1차원)** 와 **DataFrame(2차원)**  
- 데이터 탐색: `head()`, `tail()`, `shape`, `columns`, `info()`, `describe()`  
- 인덱스와 컬럼명을 자유롭게 설정 가능 (`set_index`, `reset_index`, `rename`)  
- 데이터 타입 변환은 `astype()`  
- 불린 인덱싱으로 조건을 만족하는 행 선택 가능  
- `drop()`으로 행/열을 삭제할 수 있다.

# 2. 데이터 다듬기

데이터는 항상 깨끗하지 않습니다.  
- **결측값 (missing values)**: 데이터가 비어 있는 값  
- **중복값 (duplicates)**: 동일한 데이터가 여러 번 등장  
- **이상점 (outliers)**: 다른 값들과 동떨어진 극단적인 값  

이러한 문제를 처리해야 **신뢰할 수 있는 분석**을 할 수 있습니다.  



## 2.1 결측값 처리

### 결측값 확인
- `isna()`: 값이 NaN인지 True/False 반환  
- `sum()`: 결측치 개수 세기

In [ ]:
# Titanic 데이터 예시
print(titanic.isna().sum())

### 결측값 제거
- `dropna()`: 결측치가 있는 행/열 삭제  

In [ ]:
titanic_drop = titanic.dropna(subset=["Age"])  # Age가 NaN인 행 제거
print(titanic_drop.shape)

### 결측값 대체
- `fillna(값)`: 결측치를 특정 값으로 채우기

In [ ]:
# Bike Sharing 예시: 기온/습도 결측치를 평균값이나 중간값으로 채울 수 있음  
# 평균 나이로 채우기
mean_age = titanic["Age"].mean()
titanic["Age_filled"] = titanic["Age"].fillna(mean_age)

## 2.2 중복값 처리

### 중복 확인
- `duplicated()`: 각 행이 중복인지 True/False 반환  
- `sum()`: 중복 개수 세기

In [ ]:
print(titanic.duplicated().sum())

### 중복 제거
- `drop_duplicates()`: 중복 행 제거  


In [ ]:
titanic_unique = titanic.drop_duplicates()
print(titanic_unique.shape)

## 2.3 이상점 처리

### 이상점이란?
- 데이터 분포에서 일반적인 범위에서 크게 벗어난 값  
- 분석 결과를 왜곡할 수 있으므로 확인이 필요  

### 박스플롯으로 이상점 시각화
- 박스플롯은 사분위수를 기반으로 이상치를 확인  

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x=titanic["Age"])
plt.show()






### IQR(사분위 범위) 방식으로 이상점 탐지
- IQR = Q3 - Q1  
- 하한 = Q1 - 1.5 × IQR  
- 상한 = Q3 + 1.5 × IQR  
- 범위를 벗어난 값 = 이상점  



In [ ]:
# Bike Sharing 예시: 대여량(`count`)이 너무 비정상적으로 큰 날을 탐지할 수 있음  

Q1 = titanic["Age"].quantile(0.25)
Q3 = titanic["Age"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = titanic[(titanic["Age"] < lower_bound) | (titanic["Age"] > upper_bound)]
print(outliers[["Age"]])

### ✅ 체크포인트
- **결측값**: `isna()`, `dropna()`, `fillna()`로 확인·삭제·대체할 수 있다.  
- **중복값**: `duplicated()`, `drop_duplicates()`로 처리할 수 있다.  
- **이상점**: 박스플롯과 IQR 방식을 활용해 탐지할 수 있다.  


# 3. 문자 데이터 가공

데이터 분석에서는 **문자열 처리**가 매우 자주 필요합니다.  
예를 들어, 승객 이름에서 호칭(Mr, Mrs 등)을 추출하거나, 텍스트 데이터를 정리할 때 사용됩니다.  

## 3.1 대소문자 처리
- `str.lower()` : 모두 소문자로 변환  
- `str.upper()` : 모두 대문자로 변환  
- `str.title()` : 단어의 첫 글자만 대문자로 변환  

In [ ]:
names = pd.Series([" Alice ", "BOB", "cHaRlIe "])

print(names.str.lower())   # 소문자 변환
print(names.str.upper())   # 대문자 변환
print(names.str.title())   # 첫 글자만 대문자

## 3.2 문자열 공백 제거
- `str.strip()` : 앞뒤 공백 제거  
- `str.lstrip()` : 왼쪽 공백 제거  
- `str.rstrip()` : 오른쪽 공백 제거  

In [ ]:
print(names.str.strip())

## 3.3 문자열 분리
- `str.split("구분자")` : 구분자를 기준으로 분리 → 리스트 반환  
- `str.get(인덱스)` 또는 `.str[인덱스]`로 특정 위치 추출  

In [ ]:
# Titanic 예시: 승객 이름에서 호칭(Mr, Mrs, Miss 등)을 추출  
titanic["Title"] = titanic["Name"].str.split(",").str[1].str.split(".").str[0].str.strip()
print(titanic[["Name", "Title"]].head())


## 3.4 문자열 대체
- `str.replace("찾을문자", "바꿀문자")`  

In [ ]:
city = pd.Series(["New York", "Los Angeles", "new york"])

print(city.str.replace("New York", "NYC", case=False))

## 3.5 조건 필터링 (문자열 포함 여부)
- `str.contains("문자열")`: 특정 문자가 포함된 행만 True  

In [ ]:
mask = titanic["Name"].str.contains("Smith")
print(titanic[mask][["Name", "Age"]])

### ✅ 체크포인트
- 문자열을 소문자/대문자/타이틀 케이스로 변환할 수 있다.  
- 공백 제거, 문자열 분리(split), 대체(replace) 등의 전처리 작업이 가능하다.  
- `.str.contains()`로 특정 문자를 포함한 행을 쉽게 찾을 수 있다.  
- Titanic 데이터에서는 이름(`Name`)에서 호칭 추출, 특정 성씨 검색 등에 활용할 수 있다.  


# 4. 숫자 데이터 가공

숫자 데이터는 단순 계산뿐 아니라, **정규화/표준화**, **구간화(binning)**, **사용자 정의 함수 적용** 등의 작업이 자주 필요합니다.  



## 4.1 기본 연산
DataFrame/Series는 벡터 연산이 가능하므로, 사칙연산을 바로 적용할 수 있습니다.

In [ ]:
numbers = pd.Series([10, 20, 30, 40])

print(numbers + 5)   # 모든 원소에 +5
print(numbers * 2)   # 모든 원소에 ×2
print(numbers / 10)  # 모든 원소에 ÷10

## 4.2 정규화 (Normalization)
데이터를 **0~1 사이 값**으로 바꿔 비교 가능하게 함.  

$$
x' = \frac{x - \min(x)}{\max(x) - \min(x)}
$$

In [ ]:
age = titanic["Age"].dropna()
age_norm = (age - age.min()) / (age.max() - age.min())
print(age_norm.head())

## 4.3 표준화 (Standardization)
데이터를 **평균 0, 표준편차 1**로 맞춤.  

$$
z = \frac{x - \mu}{\sigma}
$$

In [ ]:
age_std = (age - age.mean()) / age.std()
print(age_std.head())

## 4.4 구간화 (Binning)
연속형 데이터를 범주형 데이터로 변환  

- `pd.cut()` : 구간 기준으로 나누기  
- `pd.qcut()` : 분위수(quantile) 기준으로 나누기  

In [ ]:
# 나이를 연령대 구간으로 나누기
bins = [0, 12, 18, 30, 50, 80]
labels = ["Child", "Teen", "Young", "Adult", "Senior"]

titanic["AgeGroup"] = pd.cut(titanic["Age"], bins=bins, labels=labels)
print(titanic[["Age", "AgeGroup"]].head())


## 4.5 apply() 함수
복잡한 연산을 함수로 정의하고 적용 가능  

In [ ]:
def age_category(age):
    if pd.isna(age):
        return "Unknown"
    elif age < 18:
        return "Child"
    else:
        return "Adult"

titanic["AgeCategory"] = titanic["Age"].apply(age_category)
print(titanic[["Age", "AgeCategory"]].head())

### ✅ 체크포인트
- 숫자 데이터는 사칙연산으로 쉽게 벡터 연산 가능하다.  
- **정규화**는 0~1 범위로, **표준화**는 평균0/표준편차1로 맞춘다.  
- `pd.cut()`, `pd.qcut()`으로 구간화가 가능하다.  
- `apply()`를 사용하면 사용자 정의 함수를 각 값에 적용할 수 있다.  


# 5. 날짜와 시간 데이터 다루기

데이터 분석에서는 **시간 정보**가 매우 중요합니다.  
예: 주문 시간, 탑승일, 자전거 대여 일자 등.  
Pandas는 `datetime` 타입을 활용하여 날짜와 시간을 쉽게 다룰 수 있습니다.  



## 5.1 문자열을 날짜로 변환
- `pd.to_datetime()` : 문자열 → 날짜/시간 데이터 변환  

In [ ]:
bike["datetime"] = pd.to_datetime(bike["datetime"])
print(bike["datetime"].head())
print(bike["datetime"].dtype)  # datetime64[ns]

## 5.2 날짜 속성 추출
- `.dt.year`, `.dt.month`, `.dt.day`, `.dt.weekday`, `.dt.hour` 등으로 세부 요소 추출 가능

In [ ]:
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["weekday"] = bike["datetime"].dt.weekday
bike["hour"] = bike["datetime"].dt.hour

print(bike[["datetime", "year", "month", "weekday", "hour"]].head())

## 5.3 날짜를 인덱스로 설정
- `set_index()`로 datetime 컬럼을 인덱스로 지정하면 시계열 데이터 분석에 유용  

In [ ]:
bike = bike.set_index("datetime")
print(bike.head())

## 5.4 리샘플링 (Resampling)
- `resample("규칙")` : 특정 단위(일, 주, 월 등)로 데이터 묶기  
- 규칙 예시: `"D"`=일, `"W"`=주, `"M"`=월

In [ ]:
# 일별 평균 대여량
daily_mean = bike["count"].resample("D").mean()
print(daily_mean.head())

# 월별 평균 대여량
monthly_mean = bike["count"].resample("M").mean()
print(monthly_mean.head())

## 5.5 시각화와 함께 활용

In [ ]:

import matplotlib.pyplot as plt

monthly_mean.plot(kind="line", title="Monthly Average Bike Demand")
plt.ylabel("count")
plt.show()

### ✅ 체크포인트
- `pd.to_datetime()`으로 문자열을 datetime 타입으로 변환할 수 있다.  
- `.dt.year`, `.dt.month`, `.dt.weekday`, `.dt.hour` 등으로 세부 시간 정보를 추출할 수 있다.  
- datetime을 인덱스로 설정하면 시계열 분석에 유리하다.  
- `resample()`로 원하는 주기(일, 주, 월) 단위로 데이터 집계가 가능하다.  


# 6. 데이터 합치기

분석을 하다 보면 데이터를 여러 개의 테이블(파일)로 나누어 관리하는 경우가 많습니다.  
이때 Pandas의 **concat, merge, join**을 사용하여 데이터를 합칠 수 있습니다.  



## 6.1 concat()
- 여러 DataFrame을 **위아래(행)** 또는 **옆으로(열)** 단순하게 이어 붙임  
- 인덱스를 기준으로 맞춤

In [ ]:
import pandas as pd

df1 = pd.DataFrame({"ID": [1, 2, 3], "Name": ["Alice", "Bob", "Charlie"]})
df2 = pd.DataFrame({"ID": [4, 5], "Name": ["Diana", "Evan"]})

# 위아래로 합치기 (행 기준)
df_row = pd.concat([df1, df2], axis=0)
print(df_row)

# 옆으로 합치기 (열 기준)
df_col = pd.concat([df1, pd.DataFrame({"Age": [22, 35, 40]})], axis=1)
print(df_col)

## 6.2 merge()
- SQL의 JOIN처럼 **공통된 열(키)**를 기준으로 합침  
- `how` 옵션: `"inner"`, `"outer"`, `"left"`, `"right"`

In [ ]:
df_left = pd.DataFrame({"ID": [1, 2, 3], "Age": [22, 35, 58]})
df_right = pd.DataFrame({"ID": [2, 3, 4], "City": ["Seoul", "Busan", "Incheon"]})

In [ ]:
# 기본은 inner join
df_inner = pd.merge(df_left, df_right, on="ID", how="inner")
print(df_inner)

In [ ]:
# outer join
df_outer = pd.merge(df_left, df_right, on="ID", how="outer")
print(df_outer)

## 6.3 join()
- 인덱스를 기준으로 결합  
- 기본적으로 왼쪽 DataFrame의 인덱스를 기준

In [ ]:
df_left = df_left.set_index("ID")
df_right = df_right.set_index("ID")

df_join = df_left.join(df_right, how="inner")
print(df_join)

### ✅ 체크포인트
- `concat()`은 단순히 행이나 열을 이어 붙이는 방식  
- `merge()`는 SQL JOIN처럼 키를 기준으로 데이터 병합  
- `join()`은 인덱스를 기준으로 결합  


# 7. Groupby와 피벗 테이블

데이터를 그룹별로 나누어 집계하거나, 행/열 기준으로 재구조화하여 요약할 때  
**groupby()** 와 **pivot_table()** 을 사용합니다.  



## 7.1 groupby()

- 특정 컬럼을 기준으로 데이터를 그룹화한 뒤, **집계 함수**(mean, sum, count 등)를 적용
- `df.groupby("컬럼")[다른컬럼].집계함수()`

In [ ]:
# Titanic: 성별별 평균 나이
print(titanic.groupby("Sex")["Age"].mean())

# Bike Sharing: 요일별 평균 대여량
print(bike.groupby("weekday")["count"].mean())

## 7.2 여러 컬럼으로 그룹화

- 리스트 형태로 여러 컬럼을 넣으면 다중 그룹화 가능

In [ ]:
# Titanic: 성별 + 객실 등급별 평균 생존율
print(titanic.groupby(["Sex", "Pclass"])["Survived"].mean())

## 7.3 pivot_table()

- groupby와 비슷하지만, **행/열**을 동시에 지정 가능  
- Excel의 피벗 테이블과 유사  


In [ ]:
# Titanic: 성별(Pclass) × 객실 등급별 평균 생존율
pivot = titanic.pivot_table(values="Survived", index="Sex", columns="Pclass", aggfunc="mean")
print(pivot)

## 7.4 집계 함수 응용

- `agg()`를 사용하면 여러 개의 집계 함수를 한 번에 적용 가능  

In [ ]:
# Titanic: 성별별 나이의 평균과 중앙값 동시 계산
print(titanic.groupby("Sex")["Age"].agg(["mean", "median"]))

## 7.5 시각화와 함께 활용

In [ ]:
# 성별/등급별 생존율을 시각화
pivot.plot(kind="bar", title="Survival Rate by Sex and Pclass")
plt.ylabel("Survival Rate")
plt.show()

### ✅ 체크포인트
- `groupby()`는 그룹별 집계 → 단일/다중 컬럼 기준 가능  
- `pivot_table()`은 행/열을 동시에 지정하는 요약 테이블  
- `agg()`로 여러 통계치를 한 번에 계산 가능  
- groupby, pivot_table은 데이터 분석에서 패턴 파악에 필수적인 도구  


### 실습 문제

### 문제 1. 결측값 처리
Titanic 데이터의 `Age` 컬럼에서 결측치 개수를 확인하고, 평균값으로 채우세요.  

<details>
<summary>정답 보기</summary>

```python
# 결측치 개수 확인
print(titanic["Age"].isna().sum())

# 평균값으로 채우기
mean_age = titanic["Age"].mean()
titanic["Age_filled"] = titanic["Age"].fillna(mean_age)
```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 2. 중복값 제거
Titanic 데이터에서 중복된 행의 개수를 확인하고, 제거한 뒤 남은 데이터 크기를 출력하세요.  

<details>
<summary>정답 보기</summary>

```python
print(titanic.duplicated().sum())   # 중복 행 개수
titanic_unique = titanic.drop_duplicates()
print(titanic_unique.shape)
```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 3. 이상점 탐지
Titanic 데이터에서 `Fare` 컬럼의 이상점을 IQR 방식을 사용해 탐지하세요.  

<details>
<summary>정답 보기</summary>

```python
Q1 = titanic["Fare"].quantile(0.25) # 하위 25%
Q3 = titanic["Fare"].quantile(0.75) # 상위 25%
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = titanic[(titanic["Fare"] < lower_bound) | (titanic["Fare"] > upper_bound)]
print(outliers[["Fare"]].head())
```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 4. 문자열 가공
Titanic 데이터의 `Name` 컬럼에서 승객 호칭(예: Mr, Mrs, Miss)을 추출하여 새로운 컬럼 `Title`을 만드세요.  

<details>
<summary>정답 보기</summary>

```python
titanic["Title"] = titanic["Name"].str.split(",").str[1].str.split(".").str[0].str.strip()
print(titanic[["Name", "Title"]].head())
```
</details>



In [ ]:
# 여기에 작성하세요


### 문제 5. 구간화
Titanic 데이터의 `Age` 컬럼을 기준으로 나이를 `Child(0-12), Teen(13-19), Adult(20-64), Senior(65+)` 구간으로 나누어 새로운 컬럼을 추가하세요.  

<details>
<summary>정답 보기</summary>

```python
bins = [0, 12, 19, 64, 100]
labels = ["Child", "Teen", "Adult", "Senior"]

titanic["AgeGroup"] = pd.cut(titanic["Age"], bins=bins, labels=labels)
print(titanic[["Age", "AgeGroup"]].head())
```
</details>

In [ ]:
# 여기에 작성하세요
bins = [0, 12, 19, 64, 100]
labels = ["Child", "Teen", "Adult", "Senior"]

### 문제 6. 날짜 처리
Bike Sharing 데이터에서 `datetime` 컬럼을 datetime 타입으로 변환하고, `year`, `month`, `weekday` 컬럼을 추출하세요.  

<details>
<summary>정답 보기</summary>

```python
bike["datetime"] = pd.to_datetime(bike["datetime"])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["weekday"] = bike["datetime"].dt.weekday

print(bike[["datetime", "year", "month", "weekday"]].head())
```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 7. 데이터 합치기
아래 두 DataFrame을 `ID` 컬럼을 기준으로 merge하세요.  


df_left = pd.DataFrame({"ID": [1, 2, 3], "Age": [22, 35, 58]})
df_right = pd.DataFrame({"ID": [2, 3, 4], "City": ["Seoul", "Busan", "Incheon"]})


<details>
<summary>정답 보기</summary>

```python
df_merge = pd.merge(df_left, df_right, on="ID", how="inner")
print(df_merge)
```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 8. 그룹화 & 피벗 테이블
Titanic 데이터에서 성별(`Sex`)과 객실 등급(`Pclass`)별 평균 생존율을 구하고, 피벗 테이블로 요약하세요.  

<details>
<summary>정답 보기</summary>

```python
# groupby
print(titanic.groupby(["Sex", "Pclass"])["Survived"].mean())

# pivot_table
pivot = titanic.pivot_table(values="Survived", index="Sex", columns="Pclass", aggfunc="mean")
print(pivot)
```
</details>

### 문제 9. 다중 조건 불린 인덱싱
Titanic 데이터에서 Age >= 30 이고 Sex == "male" 인 행만 선택하여 Name, Age, Sex, Fare 컬럼만 보여주세요.

<details>
<summary>정답 보기</summary>

```python
mask = (titanic["Age"] >= 30) & (titanic["Sex"] == "male")
print(titanic.loc[mask, ["Name", "Age", "Sex", "Fare"]].head())
```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 10. 다중 기준 정렬
Titanic 데이터를 Age 오름차순, Fare 내림차순으로 동시에 정렬하고 상위 10행을 출력하세요.

<details>
<summary>정답 보기</summary>

```python
sorted_df = titanic.sort_values(by=["Age", "Fare"], ascending=[True, False])
print(sorted_df.head(10))
```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 11. 인덱스 설정/리셋
Titanic 데이터에서 Name을 인덱스로 설정한 후, 다시 기본 정수 인덱스로 리셋하세요.
<details>
<summary>정답 보기</summary>

```python
titanic_idx = titanic.set_index("Name")
print(titanic_idx.head())

titanic_reset = titanic_idx.reset_index()
print(titanic_reset.head())
```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 12. 카테고리형 변환
Pclass와 Sex 컬럼을 범주형(category)으로 변환하고 info()로 타입 변화를 확인하세요.

<details>
<summary>정답 보기</summary>

```python
titanic["Pclass"] = titanic["Pclass"].astype("category")
titanic["Sex"] = titanic["Sex"].astype("category")
titanic.info()
```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 13. 사용자 정의 함수 적용
Age를 이용해 성인/미성년 여부를 표시하는 IsAdult 컬럼을 생성하세요. (나이 < 18 → 0, 그 외 → 1. 결측치는 -1)

<details>
<summary>정답 보기</summary>

```python
def is_adult(age):
    if pd.isna(age):
        return -1
    return 1 if age >= 18 else 0

titanic["IsAdult"] = titanic["Age"].apply(is_adult)
print(titanic[["Age", "IsAdult"]].head())

```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 14. 분위수 기반 구간화(qcut)
Fare를 사분위수 기준 4개 구간으로 나누어 FareQuartile 컬럼을 생성하세요.

<details>
<summary>정답 보기</summary>

```python
titanic["FareQuartile"] = pd.qcut(titanic["Fare"], q=4, labels=["Q1","Q2","Q3","Q4"])
print(titanic[["Fare", "FareQuartile"]].head())

```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 15. 중복값 처리(부분 컬럼 기준)

Name과 Ticket 조합이 중복된 행을 제거하세요(첫 번째만 남김). 제거 전/후 행 수를 출력하세요.

<details>
<summary>정답 보기</summary>

```python
before = len(titanic)
titanic_dedup = titanic.drop_duplicates(subset=["Name", "Ticket"])
after = len(titanic_dedup)
print("제거 전:", before, "제거 후:", after)
```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 16. 문자열 처리(덱 정보 추출)

Cabin 결측치는 "Unknown"으로 채우고, CabinDeck(첫 문자만 추출) 컬럼을 만드세요.

<details>
<summary>정답 보기</summary>

```python
titanic["Cabin"] = titanic["Cabin"].fillna("Unknown")
titanic["CabinDeck"] = titanic["Cabin"].astype(str).str[0]
print(titanic[["Cabin", "CabinDeck"]].head())

```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 17. 시계열 리샘플링

Bike Sharing 데이터에서 datetime을 인덱스로 설정한 뒤, count의 월별 평균을 계산하세요.

<details>
<summary>정답 보기</summary>

```python
7. 시계열 리샘플링

Bike Sharing 데이터에서 datetime을 인덱스로 설정한 뒤, count의 월별 평균을 계산하세요.
```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 18. 다양한 조인 실습

아래 두 DataFrame으로 left join과 outer join 결과를 각각 만드세요.

```python
A = pd.DataFrame({"ID":[1,2,3], "City":["Seoul","Busan","Daegu"]})
B = pd.DataFrame({"ID":[2,3,4], "Income":[50,60,70]})
```

<details>
<summary>정답 보기</summary>

```python
A = pd.DataFrame({"ID":[1,2,3], "City":["Seoul","Busan","Daegu"]})
B = pd.DataFrame({"ID":[2,3,4], "Income":[50,60,70]})

left_join = pd.merge(A, B, on="ID", how="left")
outer_join = pd.merge(A, B, on="ID", how="outer")

print(left_join)
print(outer_join)
```
</details>

In [ ]:
# 여기에 작성하세요
A = pd.DataFrame({"ID":[1,2,3], "City":["Seoul","Busan","Daegu"]})
B = pd.DataFrame({"ID":[2,3,4], "Income":[50,60,70]})

### 문제 19. 그룹 집계 다중 통계

Titanic 데이터에서 Sex별 Age의 평균·중앙값, Fare의 평균·최대값을 한 번에 구하세요.

<details>
<summary>정답 보기</summary>

```python
agg_df = titanic.groupby("Sex").agg({
    "Age": ["mean", "median"],
    "Fare": ["mean", "max"]
})
print(agg_df)

```
</details>

In [ ]:
# 여기에 작성하세요

### 문제 20. 피벗 테이블 + 합계(margins)

Pclass(행) × Sex(열) 기준 Survived 평균을 피벗 테이블로 만들고, 행/열 합계를 포함하여 출력하세요.

<details>
<summary>정답 보기</summary>

```python
pivot = titanic.pivot_table(
    values="Survived",
    index="Pclass",
    columns="Sex",
    aggfunc="mean",
    margins=True,        # 합계(평균) 행/열
    margins_name="Total" # 합계 라벨명
)
print(pivot)

```
</details>

In [ ]:
# 여기에 작성하세요
pivot = titanic.pivot_table # 코드를 이어서 완성하세요

### ✅ 체크포인트
- 결측값, 중복값, 이상점은 반드시 확인 후 처리해야 한다.  
- 문자열, 숫자, 날짜 데이터를 가공하여 분석에 적합한 형태로 바꿀 수 있다.  
- `merge()`, `groupby()`, `pivot_table()`은 데이터 분석 실무에서 자주 쓰이는 핵심 기능이다.

# 정리

### 1. DataFrame 기본기
- **DataFrame, Series 탐색하기**  
  - `head()`, `tail()`, `shape`, `columns`, `info()`
- **데이터 타입 설정하기**  
  - `astype()`으로 형 변환
- **인덱스와 컬럼명 다루기**  
  - `set_index()`, `reset_index()`, `rename()`
- **다중 조건 불린 인덱싱**  
  - `(조건1) & (조건2)`, `(조건1) | (조건2)`
- **데이터 삭제하기**  
  - `drop()`으로 행/열 삭제


### 2. 데이터 다듬기
- **결측값 처리**  
  - `isna()`, `dropna()`, `fillna()`  
- **중복값 처리**  
  - `duplicated()`, `drop_duplicates()`  
- **이상점 처리**  
  - 박스플롯 활용 → IQR 방식으로 이상치 탐색 및 제거  

👉 Titanic 예시: 나이(`Age`) 결측치 처리  
👉 Bike Sharing 예시: 이상적으로 큰 대여량 탐지  


### 3. 문자 데이터 가공
- **대소문자 처리**: `str.lower()`, `str.upper()`, `str.title()`  
- **문자열 분리**: `str.split()`  
- **공백 제거**: `str.strip()`  
- **값 대체**: `str.replace()`  

👉 Titanic 예시: 승객 이름(`Name`)에서 호칭(Mr, Mrs, Miss) 추출  
👉 Gapminder 예시: 국가 이름 전처리  


### 4. 숫자 데이터 가공
- **사칙연산**: 기본 연산자 활용  
- **정규화/표준화**: (x-min)/(max-min), (x-mean)/std  
- **구간화(Binning)**: `pd.cut()`, `pd.qcut()`  
- **apply 함수**: 행/열 단위 사용자 정의 연산  

👉 Bike Sharing 예시: 대여량(`count`)을 “Low/Medium/High” 등급으로 구간화  


### 5. 날짜와 시간 데이터
- `pd.to_datetime()`으로 문자열 → 날짜 변환  
- 속성: `.dt.year`, `.dt.month`, `.dt.weekday`  
- 시계열 인덱스: `set_index(datetime)`  
- 리샘플링: `resample("D")`, `resample("M")`  

👉 Bike Sharing 예시: 요일별/월별 수요 비교  


### 6. 데이터 합치기
- **concat()**: 위/옆으로 단순 연결  
- **merge()**: SQL의 join과 비슷 (키 기준으로 병합)  
- **join()**: 인덱스 기준 결합  

👉 Titanic 예시: 승객 데이터 + 추가 데이터셋 합치기  
👉 Gapminder 예시: 인구 데이터와 GDP 데이터 병합  


### 7. Groupby와 피벗 테이블
- **groupby()**: 그룹별 집계 (예: 평균, 합계)  
- **pivot_table()**: 행/열 기준 집계 결과를 표 형태로 변환  


### ✅ 체크포인트
- Pandas DataFrame은 탐색 → 클리닝 → 가공 → 분석 → 시각화까지 한 번에 이어진다.  
- 결측값/중복값/이상점 처리는 데이터 분석의 기본기다.  
- 문자열과 숫자 데이터를 가공하는 다양한 방법이 존재한다.  
- 날짜/시간 데이터와 그룹화 기능을 활용하면 시계열 분석까지 가능하다.  
- `concat()`, `merge()`, `groupby()`, `pivot_table()`은 데이터 분석 실무에서 반드시 익혀야 하는 핵심 함수다.  

### 실전형 추가 5문제

In [ ]:
import pandas as pd # Titanic 데이터 불러오기
titanic = pd.read_csv("data/titanic.csv") 
# Bike Sharing 데이터 불러오기
bike = pd.read_csv("data/bike_sharing.csv")

### 문제 1. 성별과 생존률 관계 분석 (Titanic)
Titanic 데이터에서 성별(`Sex`)에 따른 생존률을 비교하세요.  
- 각 성별 그룹별 생존률을 계산하세요.  
- 결과를 DataFrame 형태(`sex`, `survival_rate`)로 출력하세요.  
- 어떤 성별이 더 높은 생존률을 보였는지도 분석해 보세요.

<details><summary>정답</summary>

```python
# 성별별 생존률 계산
survival_by_sex = titanic.groupby("Sex")["Survived"].mean().reset_index()
survival_by_sex.columns = ["sex", "survival_rate"]

print(survival_by_sex)

# 분석: female의 생존률이 훨씬 높음
```

</details>

In [ ]:
# 여기에 작성하세요

### 문제 2. 객실 등급과 나이의 관계 (Titanic)
Titanic 데이터에서 객실 등급(`Pclass`)별 승객의 평균 나이를 구하세요.  
- 결측치는 제외하고 계산하세요.  
- 결과를 `pclass`, `mean_age` 형태로 출력하세요.  
- 결과를 보고, 등급별 나이에 어떤 경향이 있는지 해석해 보세요.

<details><summary>정답</summary>

```python
# 결측치 제외 후 평균 나이 계산
age_by_class = titanic.groupby("Pclass")["Age"].mean().reset_index()
age_by_class.columns = ["pclass", "mean_age"]

print(age_by_class)

# 분석: 보통 상위 등급(Pclass=1)이 평균 나이가 높음
```

</details>

In [ ]:
# 여기에 작성하세요

### 문제 3. 요일별 자전거 대여량 패턴 (Bike Sharing)
Bike Sharing 데이터에서 날짜(`dteday`)를 기준으로 요일별(`weekday`) 평균 대여량(`cnt`)을 구하세요.  
- 요일은 0=일요일, 6=토요일입니다.  
- 결과를 요일 오름차순으로 출력하세요.  
- 가장 대여량이 많은 요일과 적은 요일을 분석하세요.

<details><summary>정답</summary>

```python
# 요일별 평균 대여량
bike['weekday'] = pd.to_datetime(bike['datetime']).dt.weekday

cnt_by_weekday = bike.groupby("weekday")["count"].mean().reset_index()

print(cnt_by_weekday)

# 분석: 주말(토/일)보다 평일 특정 요일(예: 목/금)에 더 많을 수도 있음
```

</details>

In [ ]:
# 여기에 작성하세요

In [ ]:
bike['weather'].unique()

### 문제 4. 날씨가 대여량에 미치는 영향 (Bike Sharing)
Bike Sharing 데이터에서 날씨 상황(`weather`)에 따라 평균 대여량(`count`)을 구하세요.  
- `weather`: 1=맑음, 2=흐림, 3=비/눈, 4=심각한 날씨로 가정
- 평균 대여량 차이를 막대그래프로 시각화하세요.  
- 날씨가 자전거 대여량에 미치는 영향에 대해 해석해 보세요.

<details><summary>정답</summary>

```python
import matplotlib.pyplot as plt

count_by_weather = bike.groupby("weather")["count"].mean().reset_index()

# 시각화
plt.bar(count_by_weather["weather"], count_by_weather["count"])
plt.xlabel("Weather Situation")
plt.ylabel("Average Rentals")
plt.title("Bike Rentals by Weather")
plt.show()

# 분석: 맑을 때 대여량이 가장 많고, 나쁠수록 급감
```

</details>

In [ ]:
# 여기에 작성하세요

### 문제 5. 다중 조건 분석 - 생존률 & 나이 (Titanic)
Titanic 데이터에서 승객을 두 그룹으로 나눠서 생존률을 비교하세요.  
- 그룹 1: 18세 이하 승객 (Children)  
- 그룹 2: 19세 이상 승객 (Adults)  
- 각 그룹을 성별(`Sex`)까지 나눠서 생존률을 계산하세요.  
- 결과를 `group`, `sex`, `survival_rate` 형태로 출력하고, 어떤 그룹이 가장 높은 생존률을 보이는지 해석하세요.

<details><summary>정답</summary>

```python
# 나이 그룹 컬럼 생성
titanic["AgeGroup"] = titanic["Age"].apply(
    lambda x: "Child" if x <= 18 else "Adult"
)

# 그룹별 + 성별 생존률
survival = titanic.groupby(["AgeGroup", "Sex"])["Survived"].mean().reset_index()
survival.columns = ["group", "sex", "survival_rate"]

print(survival)

# 분석: female children의 생존률이 가장 높음
```

</details>

In [ ]:
# 여기에 작성하세요